In [ ]:
import os
import multiprocessing
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Dataset

In [ ]:
# Define the training and validation base directories
train_dir = "/content/drive/MyDrive/model/Dataset/Train"
validation_dir = "/content/drive/MyDrive/model/Dataset/Valid"

def load_and_preprocess_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (400, 400))
    image = tf.cast(image, tf.float32)
    image = image / 255.0
    return image, label

# Hypermarater

In [ ]:
num_classes = 15
input_shape = (400, 400, 3)
Batch_size = 128
epochs = 30

In [ ]:
def cutout(img, mask_size):
    h, w, _ = img.shape
    mask_value = img.mean()

    top = np.random.randint(0 - mask_size // 2, h - mask_size)
    left = np.random.randint(0 - mask_size // 2, w - mask_size)
    bottom = top + mask_size
    right = left + mask_size

    img[top:bottom, left:right, :] = mask_value

    return img

def train_val_generators(train_dir, validation_dir):
    # TRAIN
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode="nearest",
        preprocessing_function=lambda img: cutout(img, mask_size=16)
    )

    train_generator = train_datagen.flow_from_directory(
        directory=train_dir,
        batch_size=Batch_size,
        class_mode="categorical",
        target_size=(400, 400)
    )

    # VALIDATION
    validation_datagen = ImageDataGenerator(rescale=1/255)

    validation_generator = validation_datagen.flow_from_directory(
        directory=validation_dir,
        batch_size=Batch_size,
        class_mode="categorical",
        target_size=(400, 400)
    )

    return train_generator, validation_generator


In [ ]:
train_generator, validation_generator = train_val_generators(train_dir, validation_dir)

Found 6750 images belonging to 15 classes.
Found 894 images belonging to 15 classes.


# Pretained Model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)

    for layer in base_model.layers:
      layer.trainable = False

    #tambah layer
    x = base_model.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(2048, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)


    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer=RMSprop(learning_rate= 1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

87910968/87910968 [==============================] - 5s 0us/step


# Callbacks

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.95):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

# Fit Model

In [ ]:
callbacks = myCallback()
history = model.fit(train_generator,
                        steps_per_epoch=train_generator.n // train_generator.batch_size,
                        epochs=epochs,
                        validation_data=validation_generator,
                        validation_steps=validation_generator.n // validation_generator.batch_size,
                    callbacks=callbacks)

Epoch 1/30
52/52 [==============================] - 4532s 87s/step - loss: 1.6795 - accuracy: 0.5516 - val_loss: 1.0213 - val_accuracy: 0.7930
Epoch 2/30
52/52 [==============================] - 461s 9s/step - loss: 0.9200 - accuracy: 0.7693 - val_loss: 0.7653 - val_accuracy: 0.8008
Epoch 3/30
52/52 [==============================] - 464s 9s/step - loss: 0.6639 - accuracy: 0.8247 - val_loss: 0.6092 - val_accuracy: 0.8372
Epoch 4/30
52/52 [==============================] - 464s 9s/step - loss: 0.5421 - accuracy: 0.8506 - val_loss: 0.5286 - val_accuracy: 0.8737
Epoch 5/30
52/52 [==============================] - 462s 9s/step - loss: 0.4662 - accuracy: 0.8728 - val_loss: 0.4581 - val_accuracy: 0.8633
Epoch 6/30
52/52 [==============================] - 463s 9s/step - loss: 0.4122 - accuracy: 0.8839 - val_loss: 0.3877 - val_accuracy: 0.9023
Epoch 7/30
52/52 [==============================] - 458s 9s/step - loss: 0.3672 - accuracy: 0.8993 - val_loss: 0.3429 - val_accuracy: 0.9049
Epoch 8/30


In [ ]:
!pip install tensorflowjs

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.1/85.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.9/189.9 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: flax
    Found existing installation: flax 0.6.9
    Uninstalling flax-0.6.9:
      Successfully uninstalled flax-0.6.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
statsmodels 0.13.5 requires packaging>=21.3, but you have packaging 20.9 which is incompatible.
xar

In [ ]:
import time
saved_model_path = "./my_model.h5".format(int(time.time()))

# YOUR CODE HERE
model.save(saved_model_path)

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2023-06-12 08:12:23.934700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
from google.colab import files

In [ ]:

file_a = [
"group1-shard10of25.bin",
"group1-shard11of25.bin",
"group1-shard12of25.bin",
"group1-shard13of25.bin",
"group1-shard14of25.bin",
"group1-shard15of25.bin",
"group1-shard16of25.bin",
"group1-shard17of25.bin",
"group1-shard18of25.bin",
"group1-shard19of25.bin",
"group1-shard1of25.bin",
"group1-shard20of25.bin",
"group1-shard21of25.bin",
"group1-shard22of25.bin",
"group1-shard2of25.bin",
"group1-shard3of25.bin",
"group1-shard4of25.bin",
"group1-shard5of25.bin",
"group1-shard6of25.bin",
"group1-shard7of25.bin",
"group1-shard8of25.bin",
"group1-shard9of25.bin",

"model.json"
]


for file_path in file_a:
    files.download(file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#TF LITE
export_dir = 'saved_model/1'

# YOUR CODE HERE
tf.saved_model.save(model, export_dir)

mode = "Speed"

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)# YOUR CODE HERE

# Set the optimzations
converter.optimizations = [optimization]# YOUR CODE HERE

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()# YOUR CODE HERE

In [ ]:
import pathlib
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

26315632